#   IAU Zadanie - II. fáza

### Autori: Peter Brandajsky - 50%, Frederik Duvač - 50%

# Načítanie údajov z datasetu

Dataset sme si exportli, nakonci fázy 1, kde sme spojili **connections_mean_median** a **processes_mean_median** na **imei** a **ts** stĺpcoch, čo boli naše upravené datasety s imputáciou missing values.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

dataset = pd.read_csv('dataset/dataset_phase_2.csv', sep='\t')

In [ ]:
dataset.head()

Atributy pri ktorých nebola ziadna korelacia, sme kompletne vyradili do dalsej manipulacie s datasetom.

In [ ]:
dataset = dataset[
    ['mwra', 'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana', 'c.android.youtube', 'p.android.documentsui',
     'p.system', 'p.android.chrome', 'p.android.externalstorage',
     'p.android.gm', 'p.android.packageinstaller', 'p.olauncher']]

plt.figure(figsize=(10, 8))
sns.heatmap(dataset.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Korelačná matica: mwra a prediktory', fontsize=16)
plt.show()

In [ ]:
dataset.info()

# 2.1 Realizácia predspracovania dát

## 2.1.A

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(
    dataset,
    test_size=0.2,
    random_state=42,
    stratify=dataset['mwra']  # Toto zabezpečí rovnakú distribúciu tried
)

# Kontrola distribúcie tried
print("Kontrola distribúcie tried:\n")
print("Pôvodný dataset:")
print(dataset['mwra'].value_counts(normalize=True).round(6))

print("\nTrénovacia množina:")
print(train_data['mwra'].value_counts(normalize=True).round(6))

print("\nTestovacia množina:")
print(test_data['mwra'].value_counts(normalize=True).round(6))

# Kontrola absolútnych počtov
print("\nAbsolútne počty:")
print(f"Celkový počet vzoriek: {len(dataset)}")
print(f"Počet trénovacích vzoriek: {len(train_data)} ({len(train_data) / len(dataset) * 100:.1f}%)")
print(f"Počet testovacích vzoriek: {len(test_data)} ({len(test_data) / len(dataset) * 100:.1f}%)")

## 2.1.B

#### Kontrola formátu dát

In [ ]:
print(f"Počet pozorovaní (riadkov): {len(train_data)}")
print(f"Počet atribútov (stĺpcov): {len(train_data.columns)}")
print("\nTypy dát v stĺpcoch:")
print(train_data.dtypes)

Vidime ze vsetky data mame numericke takze nepotrebujeme robit ziaden encoding.

#### Kontrola chýbajúcich hodnôt

In [ ]:
print("\nKontrola chýbajúcich hodnôt:")
missing_values = train_data.isnull().sum()
print(missing_values)

if missing_values.any():
    print("\nNahradzovanie chýbajúcich hodnôt:")
    # Pre numerické stĺpce použijeme medián
    numeric_columns = train_data.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_columns:
        if missing_values[col] > 0:
            median_value = train_data[col].median()
            train_data[col].fillna(median_value, inplace=True)
            print(f"Stĺpec {col}: nahradených {missing_values[col]} hodnôt mediánom")

Dataset neobsahuje ziadne chybajuce hodnoty

#### Kontrola a spracovanie outlierov pomocou IQR



In [ ]:
# Získanie počtu stĺpcov, ktoré chceme vizualizovať
num_columns = len(train_data.drop('mwra', axis=1).columns)

# Dynamické nastavenie počtu riadkov a stĺpcov pre mriežku grafov
cols = 2
rows = np.ceil(num_columns / cols).astype(int)  # zaokrúhlenie nahor

plt.figure(figsize=(12, 20))
for i, column in enumerate(train_data.drop('mwra', axis=1).columns):
    plt.subplot(rows, cols, i + 1)
    sns.histplot(data=train_data, x=column, kde=True)
    plt.title(f'Distribúcia {column}')
plt.tight_layout()
plt.show()

V prvej faze sme risili odstraneni outlierov a nahradenie nan hodnot priemerom alebo medianom. Data po rozdeleni ukazuju nejakych outlierov tak ich nahradime technikou winsorization, cize hranicnymi hodnotami.

In [ ]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Počet outlierov
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    print(f"\nStĺpec {column}:")
    print(f"Počet outlierov: {len(outliers)}")
    print(f"Percento outlierov: {(len(outliers) / len(df) * 100):.2f}%")

    # Nahradenie outlierov hraničnými hodnotami
    df.loc[df[column] < lower_bound, column] = lower_bound
    df.loc[df[column] > upper_bound, column] = upper_bound

    return df


# Spracovanie outlierov pre každý numerický stĺpec okrem target premennej
print("\nSpracovanie outlierov:")
for column in train_data.select_dtypes(include=['float64', 'int64']).columns:
    if column != 'mwra':
        train_data = handle_outliers(train_data, column)

print(f"\nPočet riadkov po odstránení outlierov: {len(train_data)}")


#### Kontrola vyslednych dat

In [ ]:
print("\nKontrola výsledných dát:")
print("\nZákladné štatistiky:")
print(train_data.describe())

print("\nKontrolné body:")
print(f"- Počet pozorovaní (riadkov): {len(train_data)}")
print(f"- Počet atribútov (stĺpcov): {len(train_data.columns)}")
print("- Všetky atribúty sú numerické:", all(train_data.dtypes.apply(lambda x: np.issubdtype(x, np.number))))
print("- Žiadne chýbajúce hodnoty:", not train_data.isnull().any().any())

# Získanie počtu stĺpcov, ktoré chceme vizualizovať
num_columns = len(train_data.drop('mwra', axis=1).columns)

# Dynamické nastavenie počtu riadkov a stĺpcov pre mriežku grafov
cols = 2
rows = np.ceil(num_columns / cols).astype(int)  # zaokrúhlenie nahor

plt.figure(figsize=(12, 20))
for i, column in enumerate(train_data.drop('mwra', axis=1).columns):
    plt.subplot(rows, cols, i + 1)
    sns.histplot(data=train_data, x=column, kde=True)
    plt.title(f'Distribúcia {column}')
plt.tight_layout()
plt.show()

In [ ]:
# Získanie počtu stĺpcov, ktoré chceme vizualizovať
num_columns = len(train_data.drop('mwra', axis=1).columns)

# Dynamické nastavenie počtu riadkov a stĺpcov pre mriežku grafov
cols = 2
rows = np.ceil(num_columns / cols).astype(int)  # zaokrúhlenie nahor

plt.figure(figsize=(12, 20))
for i, column in enumerate(train_data.drop('mwra', axis=1).columns):
    plt.subplot(rows, cols, i + 1)
    sns.boxplot(data=train_data, x=column)
    plt.title(f'Distribúcia {column}')
plt.tight_layout()
plt.show()

## 2.1.C - Data Scaling

### Data Normalization

In [ ]:
# Vykreslenie histogramov a Q-Q plotov
for column in train_data.drop('mwra', axis=1).columns:
    plt.figure(figsize=(12, 6))

    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(train_data[column], kde=True)
    plt.title(f'Histogram: {column}')

    # Q-Q plot
    plt.subplot(1, 2, 2)
    stats.probplot(train_data[column], dist="norm", plot=plt)
    plt.title(f'Q-Q Plot: {column}')

    plt.tight_layout()
    plt.show()

Z histogramov a qq plotov vidime ze data mame priblizne gaussian a p.launcher nie. 

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer

In [ ]:
# Stĺpce podľa typu rozdelenia
normal_columns = ['c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana',
                  'c.android.youtube', 'p.android.documentsui', 'p.system',
                  'p.android.chrome', 'p.android.externalstorage', 'p.android.gm',
                  'p.android.packageinstaller']

skewed_columns = ['p.olauncher']


Vo všeobecnosti by sa transformácia mala vykonať pred škálovaním. Hlavný dôvod je ten, že transformácia údajov (napríklad pomocou log, Power alebo Quantile Transformácie) zmení ich distribúciu. Škálovanie následne upraví dáta do požadovaného rozsahu alebo štandardného rozloženia, čo je obzvlášť dôležité pri modeloch, ktoré sú citlivé na rozsah hodnôt.

Poradie:

1. Transformácia: Najprv vykonáme transformáciu, ktorá zlepšuje symetriu alebo normalitu dát.
2. Škálovanie: Potom aplikujeme škálovanie, ktoré prispôsobí hodnoty do rovnakého rozsahu alebo na štandardnú mierku.

#### 1. Transformácia (Quantile a Power)

In [ ]:
# Quantile transformácia pre zmenu na normalovu distribuciu
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
train_data_quantile_transformed = train_data.copy()
train_data_quantile_transformed[skewed_columns] = quantile_transformer.fit_transform(train_data[skewed_columns])

In [ ]:
# Power transformácia pre približne normálne rozdelené atribúty
power_transformer = PowerTransformer(method='yeo-johnson')
train_data_power_transformed = train_data.copy()
train_data_power_transformed[normal_columns] = power_transformer.fit_transform(train_data[normal_columns])

#### 2. Škálovanie (Standard a Min-Max)

Oba typy škálovania majú svoje výhody a vhodnosť použitia závisí od zvoleného modelu a typu dát. **Min-Max Scaling** je vhodný na škálovanie do pevného rozsahu (napr. 0 až 1), zatiaľ čo **Standard Scaling** je užitočný, ak potrebujeme normalizovať atribúty do symetrického rozsahu okolo 0.

In [ ]:
# Standard Scaling pre normálne rozdelené atribúty
standard_scaler = StandardScaler()
train_data_standard_scaled = train_data_power_transformed.copy()  # použijeme transformované údaje z Power Transformácie
train_data_standard_scaled[normal_columns] = standard_scaler.fit_transform(train_data_power_transformed[normal_columns])


In [ ]:
# Min-Max Scaling pre silne šikmý atribút
minmax_scaler = MinMaxScaler()
train_data_minmax_scaled = train_data_quantile_transformed.copy()  # použijeme transformované údaje z Quantile Transformácie
train_data_minmax_scaled[skewed_columns] = minmax_scaler.fit_transform(train_data_quantile_transformed[skewed_columns])

In [ ]:
print("Standard Scaling on Power Transformed data for normal columns:")
print(train_data_standard_scaled[normal_columns].head())

print("\nMin-Max Scaling on Quantile Transformed skewed column:")
print(train_data_minmax_scaled[skewed_columns].head())


In [ ]:
# Získanie počtu stĺpcov, ktoré chceme vizualizovať
num_columns = len(train_data.drop('mwra', axis=1).columns)

# Dynamické nastavenie počtu riadkov a stĺpcov pre mriežku grafov
cols = 4
rows = np.ceil(num_columns / cols).astype(int)  # zaokrúhlenie nahor

plt.figure(figsize=(12, 12))
for i, column in enumerate(train_data.drop('mwra', axis=1).columns):
    plt.subplot(rows, cols, i + 1)
    sns.histplot(data=train_data, x=column, kde=True)
    plt.title(f'Distribúcia {column}')
plt.tight_layout()
plt.show()

## 2.1.D - Zhrnutie

1. **Poradie:** Najskôr sme aplikovali transformáciu (Quantile alebo Power) a potom škálovanie.
2. **Výber transformácií a škálovania:**
    - Standard Scaling je ideálny pre atribúty, ktoré majú približne normálne rozdelenie, zatiaľ čo Min-Max Scaling je vhodný pre atribúty s veľkým rozsahom hodnôt a šikmosťou.
    - Quantile Transformácia a Power Transformácia (Yeo-Johnson) zlepšujú symetriu a normalitu dát.

Tento prístup pripraví dáta na efektívne využitie v modeloch strojového učenia a pomôže zlepšiť stabilitu modelov, najmä pri algoritmoch citlivých na rozsah hodnôt.

# 2.2 Výber atribútov pre strojové učenie

## 2.2.A

Použil som 4 rôzne techniky na určenie dôležitosti features:

1. Korelačná analýza

    Meria lineárny vzťah medzi každým feature a cieľovou premennou<br>
    Výhody: Jednoduchá interpretácia<br>
    Nevýhody: Zachytí len lineárne vzťahy


2. Mutual Information

    Meria ako veľa informácie poskytuje feature o cieľovej premennej<br>
    Výhody: Zachytí aj nelineárne vzťahy<br>
    Nevýhody: Hodnoty sú ťažšie interpretovateľné


3. ANOVA F-value

    Testuje štatistickú významnosť rozdielov medzi skupinami<br>
    Výhody: Štatisticky podložené<br>
    Nevýhody: Predpokladá normálne rozdelenie


4. Random Forest Feature Importance

    Meria dôležitosť features na základe ich vplyvu na presnosť modelu<br>
    Výhody: Zachytí interakcie medzi features<br>
    Nevýhody: Môže byť ovplyvnená korelovanými features

In [ ]:
# Rozdelenie na features a target
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif

X = train_data.drop('mwra', axis=1)
y = train_data['mwra']

# 1. Korelačná analýza
print("1. Korelačná analýza:")
correlation_with_target = X.apply(lambda x: x.corr(y))
print("\nKorelácia s cieľovou premennou:")
print(correlation_with_target.sort_values(ascending=False))

# Vizualizácia korelácií
plt.figure(figsize=(10, 6))
correlation_with_target.sort_values().plot(kind='bar')
plt.title('Korelácia features s cieľovou premennou')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Mutual Information
print("\n2. Mutual Information Score:")
mi_scores = mutual_info_classif(X, y)
mi_scores = pd.Series(mi_scores, index=X.columns)
print(mi_scores.sort_values(ascending=False))

plt.figure(figsize=(10, 6))
mi_scores.sort_values().plot(kind='bar')
plt.title('Mutual Information Scores')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 3. ANOVA F-value
print("\n3. ANOVA F-value:")
f_scores = f_classif(X, y)[0]
f_scores = pd.Series(f_scores, index=X.columns)
print(f_scores.sort_values(ascending=False))

plt.figure(figsize=(10, 6))
f_scores.sort_values().plot(kind='bar')
plt.title('ANOVA F-scores')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 4. Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
rf_importance = pd.Series(rf.feature_importances_, index=X.columns)
print("\n4. Random Forest Feature Importance:")
print(rf_importance.sort_values(ascending=False))

plt.figure(figsize=(10, 6))
rf_importance.sort_values().plot(kind='bar')
plt.title('Random Forest Feature Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Porovnanie výsledkov všetkých metód
results = pd.DataFrame({
    'Correlation': abs(correlation_with_target),
    'Mutual_Information': mi_scores,
    'ANOVA_F_Score': f_scores,
    'RF_Importance': rf_importance
})

# Normalizácia hodnôt pre lepšie porovnanie
scaler = StandardScaler()
results_normalized = pd.DataFrame(
    scaler.fit_transform(results),
    columns=results.columns,
    index=results.index
)

# Vizualizácia porovnania

plt.figure(figsize=(12, 6))
results_normalized.plot(kind='bar')
plt.title('Porovnanie dôležitosti features podľa rôznych metód')
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
print("\nSúhrnné poradie features podľa všetkých metód:")

# Priemerné normalizované skóre
average_importance = results_normalized.mean(axis=1).sort_values(ascending=False)
print(average_importance)

# Vytvorenie heatmapy pre lepšiu vizualizáciu
plt.figure(figsize=(10, 6))
sns.heatmap(results_normalized.T, annot=True, cmap='YlOrRd')
plt.title('Heatmapa dôležitosti features')
plt.tight_layout()
plt.show()

## 2.2.B

In [ ]:
average_importance = results.mean(axis=1).sort_values()

# Vizualizácia
plt.figure(figsize=(10, 6))
average_importance.plot(kind='barh')
plt.title('Dôležitosť features (väčšia hodnota = väčšia dôležitosť)')
plt.xlabel('Priemerná dôležitosť')
plt.tight_layout()
plt.show()

## 2.2.C

### 1. Korelačná analýza
Dôvod výberu: 
- Jednoduchá a rýchla metóda na identifikáciu lineárnych vzťahov
- Hodnoty od -1 do 1 sú ľahko interpretovateľné
- Umožňuje odhaliť priame aj nepriame lineárne závislosti
<br><br>
- Výhody:
   - Rýchly výpočet
   - Ľahká interpretácia
- Nevýhody:
   - Zachytí len lineárne vzťahy
   - Citlivá na outliery

### 2. Mutual Information
Dôvod výberu:
- Meria všeobecnú závislosť (aj nelineárnu)
- Vhodná pre klasifikačné úlohy
- Nezávislá na škálovaní dát
<br><br>
- Výhody:
   - Zachytí aj nelineárne vzťahy
   - Robustná voči outlierom  
- Nevýhody:
   - Náročnejšia interpretácia hodnôt
   - Výpočtovo náročnejšia

### 3. ANOVA F-value
Dôvod výberu:
- Štatisticky podložená metóda
- Testuje rozdiely medzi skupinami
- Vhodná pre klasifikačné úlohy
<br><br>
- Výhody:
   - Štatisticky robustná
- Nevýhody:
   - Predpokladá normálne rozdelenie
   - Citlivá na veľkosť vzorky

### 4. Random Forest Feature Importance
Dôvod výberu:
- Založená na reálnom modeli strojového učenia
- Berie do úvahy interakcie medzi premennými
- Robustná voči outlierom a chýbajúcim hodnotám
<br><br>
- Výhody:
   - Zachytí komplexné vzťahy
   - Priamo súvisí s predikčnou silou
- Nevýhody:
   - Výpočtovo náročná
   - Môže byť ovplyvnená korelovanými features

### Normalizácia a porovnanie
Dôvod použitia StandardScaler:
- Rôzne metriky majú rôzne škály
- Normalizácia umožňuje priame porovnanie
- Štandardizované skóre sú ľahšie porovnateľné

Dôvod použitia viacerých vizualizácií:
1. Bar plots pre jednotlivé metódy:
   - Prehľadné zobrazenie poradia
   - Ľahko čitateľné jednotlivé hodnoty

2. Heatmapa:
   - Kompaktné zobrazenie všetkých metód
   - Farebné kódovanie pre rýchlu orientáciu
   - Možnosť vidieť vzory medzi metódami

3. Priemerná dôležitosť:
   - Sumarizácia všetkých metód
   - Konečné poradie features
   - Jednoduchá interpretácia

# 2.3 Replikovateľnosť predspracovania

## 2.3.A & 2.3.B

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, StandardScaler, MinMaxScaler, FunctionTransformer
import pandas as pd

# Definícia stĺpcov
normal_columns = [
    'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana',
    'c.android.youtube', 'p.android.documentsui', 'p.system',
    'p.android.chrome', 'p.android.externalstorage', 'p.android.gm',
    'p.android.packageinstaller'
]
skewed_column = ['p.olauncher']

X_train = train_data.drop(columns=['mwra'])
y_train = train_data['mwra']
X_test = test_data.drop(columns=['mwra'])
y_test = test_data['mwra']

# Krok 1: Quantile Transformácia pre šikmý stĺpec
quantile_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (QuantileTransformer(output_distribution="normal", random_state=42), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 2: Power Transformácia pre normálne stĺpce
power_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (PowerTransformer(method='yeo-johnson', standardize=True), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 3: Standard Scaling pre normálne stĺpce
standard_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (StandardScaler(), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 4: Min-Max Scaling pre šikmý stĺpec
minmax_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (MinMaxScaler(), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Vytvorenie hlavnej Pipeline s každým krokom zvlášť
pipeline = Pipeline(steps=[
    ('quantile', quantile_transformer),
    ('power', power_transformer),
    ('scaler', standard_scaler),
    ('minmax', minmax_scaler),
], verbose=True)

# Aplikácia pipeline na trénovacie dáta bez 'mwra'
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

# Spojenie transformovaných dát s cieľovým stĺpcom 'mwra'
train_transformed = pd.concat([y_train.reset_index(drop=True), X_train_transformed], axis=1)
test_transformed = pd.concat([y_test.reset_index(drop=True), pd.DataFrame(X_test_transformed, columns=X_test.columns)], axis=1)

train_transformed

In [ ]:
test_transformed

# Záver

1. **Replikovateľnosť predspracovania:** Definovaním predspracovateľskej pipeline sme zabezpečili, že trénovacia aj testovacia množina prejdú rovnakými transformáciami a škálovaním, čo zaručuje konzistentnosť a umožňuje jednoduché opakovanie procesu na ďalších dátach.
2. **Výhoda použitia sklearn.pipeline:** Pipeline nám umožňuje prehľadný a ľahko udržiavateľný kód, ktorý je pripravený na integráciu do modelovacieho procesu a opakovateľný pre nové dáta.

#### Export

In [ ]:
train_transformed.to_csv('dataset/train_transformed.csv', sep='\t', index=False, encoding='utf-8')
test_transformed.to_csv('dataset/test_transformed.csv', sep='\t', index=False, encoding='utf-8')

train_data.to_csv('dataset/train_data.csv', sep='\t', index=False, encoding='utf-8')
test_data.to_csv('dataset/test_data.csv', sep='\t', index=False, encoding='utf-8')